# Changing the system model

In Ocelot, the system model is defined by a list of transforming functions.

In the current version, this looks something like this:

    default_configuration = [
        variable_names_are_unique,
        # There are a *lot* of missing mandatory properties
        # No point adding them to this report
        # ensure_mandatory_properties,
        validate_markets,
        fix_ecoinvent_parameters,
        pv_cleanup,
        cleanup_activity_links,
        manage_activity_links,
        handle_waste_outputs,
        cutoff_allocation,
        drop_rp_activity_links,
        link_markets,
        # extrapolate to database reference year
        # normalize_reference_production_amount
        # final output processing
    ]

When we run the system model, we do something like this:

    system_model(path_to_data)
    
Running the system model like this will use the default configuration. We can specify a different configuration like this:

    system_model(path_to_data, another_list_of_functions)
    
In this notebook, we will develop a silly system model, and show how to define this other list of functions.

In [1]:
from ocelot import default_configuration, system_model

## The default system model configuration

Our default configuration is a list of functions as objects:

In [2]:
default_configuration

[<function ocelot.transformations.parameterization.validation.variable_names_are_unique>,
 <function ocelot.transformations.cutoff.cleanup.drop_rp_activity_links>,

This isn't that useful, as we can't see what the names are. But we can test for equality.

## Finding a specific part of the default system model

In our silly system model, we want to replace the allocation step. The allocation collection is called `cutoff_allocation`. We can import it and test where it is:

In [3]:
from ocelot.transformations.cutoff import cutoff_allocation

In [4]:
for func in default_configuration:
    print(func == cutoff_allocation)

False
False
False
False
False
False
False
True
False
False


We can also find the index in the list of functions:

In [5]:
allocation_function_index = default_configuration.index(cutoff_allocation)
allocation_function_index

7

## Creating new allocation functions

Let's handle allocation in a completely non-sensical way. Here are some rules which will always produce a single reference product:

* Delete all byproducts
* If there are more than one reference product, choose the first one that we find

It should be clear, but this **is not a real system model**. Don't use it for real.

We split this into two transforming functions:

First, to delete all byproducts, we use the function [nonreference product](https://github.com/OcelotProject/Ocelot/blob/master/ocelot/transformations/utils.py#L236). Note that this function modifies the exchange in place, so we don't need to return it.

In [6]:
from ocelot.transformations.utils import nonreference_product

def delete_all_byproducts(data):
    """Handle allocatable byproducts by deleting them all."""
    all_byproducts = (exc for ds in data for exc in ds['exchanges'] if exc['type'] == 'byproduct')
    for exc in all_byproducts:
        nonreference_product(exc)
    return data

Next, do the same thing to all reference production exchanges past the first one we find:

In [7]:
def there_can_be_only_one(data):
    """Zero out every reference production exchange past the first one in a dataset"""
    for ds in data:
        rp_iterator = (exc for exc in ds['exchanges'] if exc['type'] == 'reference product')
        found = False  # Reset for each dataset
        for exc in rp_iterator:
            if not found:
                # Don't do anything to the first reference production exchange
                found = True
                continue
            else:
                # This is the second (or higher) reference production exchange
                # We delete these
                nonreference_product(exc)
    return data

And we can define our allocation method as a list of these two functions:

In [8]:
our_allocation = [delete_all_byproducts, there_can_be_only_one]

## Replacing the allocation in the system model

We know the index in the list of system model functions, so we can just replace it by using this index:

In [9]:
default_configuration[allocation_function_index] = our_allocation

Let's test to make sure this worked:

In [10]:
default_configuration

[<function ocelot.transformations.parameterization.validation.variable_names_are_unique>,
 [<function __main__.delete_all_byproducts>,
  <function __main__.there_can_be_only_one>],
 <function ocelot.transformations.cutoff.cleanup.drop_rp_activity_links>,

And we can now run this system model:

(You will need to change the data path to fit your computer)

In [11]:
output_fp, new_data = system_model(
    "/Users/cmutel/Documents/LCA Documents/Ecoinvent/3.2/unlinked/", 
    default_configuration
)

Starting Ocelot model run
Using cached ecospold2 data
Opening log file at: /Users/cmutel/Library/Application Support/Ocelot/model-runs/c054d50497c241e6b097cc0a275615fe/report.log.json
Applying transformation variable_names_are_unique
Applying transformation ensure_markets_only_have_one_reference_product
Applying transformation ensure_markets_dont_consume_their_ref_product
Applying transformation fix_specific_ecoinvent_issues
Applying transformation replace_implicit_references
Applying transformation fix_known_bad_formula_strings
Applying transformation lowercase_all_parameters
Applying transformation fix_math_formulas
Applying transformation replace_reserved_words
Applying transformation delete_unparsable_formulas
Applying transformation ensure_production_exchanges_have_production_volume
Applying transformation add_pv_to_allocatable_byproducts
Applying transformation create_pv_parameters
Applying transformation remove_consequential_exchanges
Applying transformation drop_rp_activity_lin

There is a of stuff in this output, but if you search you will find our functions, and see that they have been applied.